In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import re
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

### Roller Coasters
source: https://www.ultimaterollercoaster.com/coasters/browse/states

In [2]:
url = "https://www.ultimaterollercoaster.com/coasters/browse/states"
response = requests.get(url)
soup = bs(response.text, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html>
 <head>
  <title>
   Roller Coasters Listed by US State | Where to Coast™
  </title>
  <meta content="Browse Roller Coasters by state. Find out Where to Coast with the Ultimate Rollercoaster database of roller coasters and amusement parks." name="description"/>
  <meta content="roller coasters, rollercoasters, roller, coaster, coasters, location, state, geographic location" name="keywords"/>
  <link href="/css/style.css" rel="stylesheet" type="text/css"/>
  <link href="/css/nav.css" rel="stylesheet" type="text/css"/>
  <link href="http://fonts.googleapis.com/css?family=Droid+Sans" rel="stylesheet" type="text/css"/>
  <link crossorigin="anonymous" href="https://use.fontawesome.com/releases/v5.4.1/css/all.css" integrity="sha384-5sAR7xN1Nv6T6+dT2mhtzEpVJvfS3NScPQTrOxhwjIuvcA67KV2R5Jz6kr4abQsz" rel="stylesheet" type="text/css"/>
  <link href="/apple-touch-icon.png" rel="apple-touch-icon"/>
  <!--[if IE 7]><link rel="stylesheet" type="text/css" href="/css/IE7styles.cs

In [3]:
# Create State List
states= soup.find_all('h3', class_="new")
state_list=[]
for state in states:
    st=state.text.strip()
    state_list.append(st)
print(state_list)

['Alabama', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Florida', 'Georgia', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin']


In [4]:
# Get count of parks by counting number of list entries per state
parks= soup.find_all('ul', class_="tpList")
coaster_ct=[]
for park in parks:
    pk=len(park.find_all('li'))
    coaster_ct.append(pk)

print(coaster_ct)

[9, 6, 7, 117, 19, 9, 65, 28, 7, 31, 16, 10, 4, 14, 7, 8, 26, 17, 10, 18, 1, 29, 1, 4, 8, 9, 62, 7, 59, 19, 54, 9, 5, 67, 4, 2, 12, 56, 11, 31, 12, 4, 15]


In [5]:
#Create new DF
parks_df=pd.DataFrame({'State': state_list, 'Coaster_ct':coaster_ct})
parks_df.head()

,State,Coaster_ct
0,Alabama,9
1,Arizona,6
2,Arkansas,7
3,California,117
4,Colorado,19


In [6]:
# Rank them (More is better so ascending = False)
parks_df['coaster_rank'] = parks_df['Coaster_ct'].rank(ascending = False, method='min', na_option='bottom').astype(int)
parks_df.head()

,State,Coaster_ct,coaster_rank
0,Alabama,9,25
1,Arizona,6,35
2,Arkansas,7,31
3,California,117,1
4,Colorado,19,13


### Craft Breweries

source: https://www.brewersassociation.org/statistics-and-data/state-craft-beer-stats/

In [7]:
url = "https://www.brewersassociation.org/statistics-and-data/state-craft-beer-stats/"
response = requests.get(url)
soup = bs(response.text)
print(soup.prettify())

<!DOCTYPE html>
<!--[if IE 7 ]> <html lang="en-US" class="no-js ie7"> <![endif]-->
<!--[if IE 8 ]> <html lang="en-US" class="no-js ie8"> <![endif]-->
<!--[if IE 9 ]> <html lang="en-US" class="no-js ie9"> <![endif]-->
<!--[if (gt IE 9)|!(IE)]><!-->
<html class="no-js" lang="en-US">
 <!--<![endif]-->
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=5, shrink-to-fit=no" name="viewport"/>
  <meta content="chrome=1, IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="text/html" http-equiv="Content-Type"/>
  <link href="http://gmpg.org/xfn/11" rel="profile"/>
  <link href="https://www.brewersassociation.org/wp-content/themes/ba2019/images/favicon.ico" rel="shortcut icon"/>
  <!-- OneTrust Cookies Consent Notice start for brewersassociation.org -->
  <script src="https://cdn.cookielaw.org/consent/25dabf7b-3194-4423-a228-d1554a048f2b/OtAutoBlock.js" type="text/javascript">
  </script>
  <script charset="UTF-8" data-domain-script="25dabf

In [8]:
# Find the state entries
states=soup.find_all('div', class_='stat-container')

#empty lists for scraped data to be appended to
statename=[] 
brewery_ct=[]
ct_rank=[]
mil_impact=[]
econ_rank=[]
barrels=[]
bar_rank=[]

for state in states:
    
    name = state.find('div', id='state-header').text.strip()
    clnname=name.rsplit("\n")[0]
    
    #webpage is not a table but results are nested using same classes 
    #have to call id then class
    per_state = state.find('div', id='per-state')
    brew = per_state.find('span', class_='count').text.strip()
    brew_rank = per_state.find('span', class_='rank').text.strip()
    #strip out just numbers from brew_rank
    brnum = re.sub("[^0-9]", "", brew_rank)
    
    econ = state.find('div', id='economic-impact')
    impact = econ.find('span', class_='count').text.strip()
    impnum = re.sub("[^0-9]", "", impact)
    e_rank = econ.find('span', class_='rank').text.strip()
    ernum = re.sub("[^0-9]", "", e_rank)
    
    prod = state.find('div', id='production')
    barr = prod.find('span', class_='count').text.strip()
    barrnum = re.sub("[^0-9]", "", barr)
    b_rank = prod.find('span', class_='rank').text.strip()
    brnum = re.sub("[^0-9]", "", b_rank)
    # Append each sta
    statename.append(clnname)
    brewery_ct.append(brew)   
    ct_rank.append(brnum)
    mil_impact.append(impnum)
    econ_rank.append(ernum)
    barrels.append(barrnum)
    bar_rank.append(brnum)
    
    


In [9]:
dataset=statename, brewery_ct, ct_rank, mil_impact, econ_rank, barrels, bar_rank
beer_df= pd.DataFrame({'State' : statename, 
                      'brewery_count' : brewery_ct, 
                      'brew_count_rank' : ct_rank, 
                      'brewery_econ_impact' : mil_impact, 
                      'brew_econ_impact_rank' : econ_rank, 
                      'brewery_production' : barrels, 
                      'brew_production_rank' : bar_rank})
beer_df.head()

,State,brewery_count,brew_count_rank,brewery_econ_impact,brew_econ_impact_rank,brewery_production,brew_production_rank
0,Alabama,52,39,777,28,88683,39
1,Alaska,53,31,254,47,166731,31
2,Arizona,121,26,1209,22,215051,26
3,Arkansas,46,45,496,37,51670,45
4,California,931,1,9031,1,3499182,1


### Art Museums by State
Source :http://www.artcyclopedia.com

In [10]:
# import list of states to create forloop with
states=pd.read_csv('csv_s/csvData-2.csv', usecols=['State'])
# because the website lists Washington DC rather than District of Columbia need to change 
# that specific item in list to make scraping loop capture all
states.at[8,'State']='washington-dc'
states.head()

,State
0,Alabama
1,Alaska
2,Arizona
3,Arkansas
4,California


In [11]:
#declare state as 'State' column from df imported above
state = states['State']
#empty list to capture values
museum_ct=[]

for name in state:
    #loop will go through each state name, filling it in the url 
    #then visit each page
    url = f"http://www.artcyclopedia.com/museums/art-museums-in-usa-{name}.html"
    response = requests.get(url)
    soup = bs(response.text, "html.parser")
    #declare results as one item to set range of below loop
    results=soup.find_all('div', id="mainpage")
    
    for row_num in range(len(results)):
        #because the page is very simple, will just pick from the soup
        #first find the third table 
        count=soup.find_all('table')[2]
        #there is a link for each listed museum in the table
        #find the 'a' tag and count them
        museums=len(count.find_all("a"))
        #append the count to list
        museum_ct.append(museums)

In [12]:
#create df of counts
counts=pd.DataFrame(museum_ct).rename(columns={0: 'art_museum_ct'})
counts.head()


,art_museum_ct
0,10
1,3
2,13
3,4
4,96


In [15]:
#combine state name to counts
art_df= states.join(counts)
# rank them (More = better so ascending = False)
art_df['art_museum_rank'] = art_df['art_museum_ct'].rank(ascending = False, method='min', na_option='bottom').astype(int)
#change DC back to keep continuity
art_df.at[8,'State']='District of Columbia'
art_df

,State,art_museum_ct,art_museum_rank
0,Alabama,10,25
1,Alaska,3,45
2,Arizona,13,19
3,Arkansas,4,42
4,California,96,2
5,Colorado,10,25
6,Connecticut,18,14
7,Delaware,5,41
8,District of Columbia,22,8
9,Florida,49,3


### Symphonies & Orchestra by State
source: https://www.wikiwand.com/en/List_of_symphony_orchestras_in_the_United_States

In [16]:
#thie page the data is inside JS so have to use chromedriver
url = "https://www.wikiwand.com/en/List_of_symphony_orchestras_in_the_United_States"
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)




====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/Users/michaelraines/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache
/var/folders/qc/_ykx45d102d0nr51ysx4tb4h0000gn/T/ipykernel_73108/491326022.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [17]:
soup = bs(driver.page_source, 'lxml')
print(soup.prettify())

<html class="no-js ng-scope" lang="en" ng-app="wikiwand" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
 <head>
  <style type="text/css">
   @charset "UTF-8";[ng\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide:not(.ng-hide-animate){display:none !important;}ng\:form{display:block;}
  </style>
  <!--header generic -->
  <meta charset="utf-8"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="//wikiwand-19431.kxcdn.com/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <link href="//wikiwand-19431.kxcdn.com/img/wikiwand_icon_apple.png" rel="apple-touch-icon"/>
  <link href="https://www.wikiwand.com/en/List_of_symphony_orchestras_in_the_United_States" rel="canonical"/>
  <!--this is the original url for google fonts-->
  <link href="//fonts.googleapis.com/css?family=Lato:300,400,700,300i

In [18]:
# take article summary from the page
states=soup.find_all('section', class_="article_content" )
#drop intro and summary sections from response
states=states[1:51]
#example response for 1 state
states[38]

<section class="article_content" id="section_Puerto_Rico"><h2>
<span class="mw-headline" id="Puerto_Rico">Puerto Rico</span><a class="edit_section tooltip_top tooltipstered" href="https://en.wikipedia.org/w/index.php?title=List_of_symphony_orchestras_in_the_United_States&amp;action=edit&amp;section=39" onclick="event.stopPropagation()" target="_blank"></a>
</h2>
<link href="mw-data:TemplateStyles:r998391716" rel="mw-deduplicated-inline-style"/><div class="div-col" style="column-width: 25em;">
<ul><li><a class="int-link" href="/en/Puerto_Rico_Symphony_Orchestra" title="Puerto Rico Symphony Orchestra">Puerto Rico Symphony Orchestra</a></li></ul>
</div></section>

In [19]:
#Empty lists to add responses to
state_list=[]
orch_ct=[]

for state in states:
    #find the name
    name=state.find('span', class_='mw-headline').text.strip()
    state_list.append(name)
    #count the items in list
    count=len(state.find_all('li'))
    orch_ct.append(count)

In [20]:
#Create new DF
orch_df=pd.DataFrame({'State': state_list, 'orchestra_ct':orch_ct})
# drop Puerto Rico from Dataset
orch_df=orch_df[orch_df['State'] != "Puerto Rico"]
orch_df.head()

,State,orchestra_ct
0,Alabama,5
1,Alaska,2
2,Arizona,3
3,Arkansas,3
4,California,31


In [21]:
# rank them (More = better so ascending = False)
orch_df['orch_rank'] = orch_df['orchestra_ct'].rank(ascending = False, method='min', na_option='bottom').astype(int)
orch_df.head()

,State,orchestra_ct,orch_rank
0,Alabama,5,18
1,Alaska,2,34
2,Arizona,3,27
3,Arkansas,3,27
4,California,31,1


### National Historic Landmarks
source: https://www.nps.gov/subjects/nationalhistoriclandmarks/list-of-nhls-by-state.htm

In [22]:
url='https://www.nps.gov/subjects/nationalhistoriclandmarks/list-of-nhls-by-state.htm'
response = requests.get(url)
soup = bs(response.text, 'html.parser')
# print(soup.prettify())

In [23]:
states=soup.find_all('h2')
#drop out the territories which are also listed on page
states=states[0:51]
#exampl of one state response
states[22]

<h2>Michigan (43)</h2>

In [24]:
#empty list for holding data
state_list= []
hist_ct=[]

for state in states:
    st_name= state.text.strip()
    # Everything before last "word" is state name, so strip at last space
    name=st_name.rsplit(' ', 1)[0]
    # first item in array is the name
    state_list.append(name)

    #second item in array is the number of sites
    count=st_name.rsplit(' ', 1)[1]
    regex = r'|".+?"|\w+'
    #regex our opening space, opening perens and take the 3rd in result array which is the number
    num_count=re.findall(regex, count)[2]
    hist_ct.append(num_count)
    
    

In [25]:
# make new df 
hist_df=pd.DataFrame({'State': state_list, 'Hist_site_ct':hist_ct})
# rank them (More = better so ascending = False)
hist_df['hist_rank'] = hist_df['Hist_site_ct'].rank(ascending = False, method='min', na_option='bottom').astype(int)
hist_df.head()

,State,Hist_site_ct,hist_rank
0,Alabama,39,23
1,Alaska,50,11
2,Arizona,47,14
3,Arkansas,17,42
4,California,147,47


### Professional Sports (multiple sports across all levels)
source: https://en.wikipedia.org/wiki/List_of_professional_sports_teams_in_the_United_States_and_Canada

In [26]:
url='https://en.wikipedia.org/wiki/List_of_professional_sports_teams_in_the_United_States_and_Canada'
sports=pd.read_html(url)

In [27]:
# the last five tables from the Ultimate Frisbee section are formatted poorly
# drop these tables
sports=sports[0:64]
#example table 
sports[1]

,League,Division,Team,City,Home park
0,National,East,Atlanta Braves,"Cumberland, Georgia",Truist Park
1,National,East,Miami Marlins,"Miami, Florida",LoanDepot Park
2,National,East,New York Mets,New York City (Queens),Citi Field
3,National,East,Philadelphia Phillies,"Philadelphia, Pennsylvania",Citizens Bank Park
4,National,East,Washington Nationals,"Washington, D.C.",Nationals Park
5,National,Central,Chicago Cubs,"Chicago, Illinois (North Side)",Wrigley Field
6,National,Central,Cincinnati Reds,"Cincinnati, Ohio",Great American Ball Park
7,National,Central,Milwaukee Brewers,"Milwaukee, Wisconsin",American Family Field
8,National,Central,Pittsburgh Pirates,"Pittsburgh, Pennsylvania",PNC Park
9,National,Central,St. Louis Cardinals,"St. Louis, Missouri",Busch Stadium


In [28]:
sport_list=[]

for table in sports:
    # bc of formatting on page, short "tables" are extra info about the league or 
    # future plans so skip these
    if len(table) > 5:
        # a few leagues don't have normalized city listings, so skip
        if "City" in table:
            # each entry in table, loop over the length of that table  
            for x in range(len(table)):
                #find the entry in the city column
                city= table['City']
                #the ones that are city, state split at the comma
                try: 
                    state= city.str.rsplit(r', ')
                    cln_state= re.sub(r"\((.*?)\)", "", state[x][1])
                #there are some oddballs where a city part is tacked onto end in parens
                #strip that out
                except:
                    state=city[x]
                    cln_state= re.sub(r"\((.*?)\)", "", state)
                sport_list.append(cln_state)           
        else:
             print("no city")
    else:
        print("too short")

too short
too short
too short
too short
too short
no city
no city
no city
no city
too short
too short
too short
too short
too short
too short
too short
too short


In [29]:
# create DataFrame
sports=pd.DataFrame(sport_list).rename(columns={0:'State'})
# for consistancy, rename some values and clean up 
sports['State'] =sports['State'].replace({'New York City': 'New York', 
                                              'New York City ': 'New York',
                                              'D.C.': 'District of Columbia', 
                                              'New YorkBridgeport': 'New York' }).str.rstrip() 



In [30]:
#create new by of groupby state capturing the size (ie- number of teams)
sports_df=sports.groupby(['State']).size().reset_index(name='sport_ct')
# Rank it
sports_df['sport_rank'] = sports_df['sport_ct'].rank(ascending = False, method='min', na_option='bottom').astype(int)
sports_df.head()

,State,sport_ct,sport_rank
0,Alabama,7,34
1,Alberta,8,29
2,Arizona,15,11
3,Arkansas,2,49
4,British Columbia,9,25


In [32]:
# because this also includes canadian states, filter on our master CSV 
states=pd.read_csv('csv_s/csvData-2.csv', usecols=['State'])
us_sports_df = sports_df.merge(states, on=['State'], how='inner')
us_sports_df

,State,sport_ct,sport_rank
0,Alabama,7,34
1,Arizona,15,11
2,Arkansas,2,49
3,California,69,1
4,Colorado,15,11
5,Connecticut,7,34
6,Delaware,4,42
7,District of Columbia,8,29
8,Florida,38,4
9,Georgia,18,10


### Merge all DF's 

In [34]:
#set index of each df as state
dfs = [df.set_index(['State']) for df in [parks_df, beer_df, art_df, orch_df, hist_df, us_sports_df]]
 
nan_value = 0

# join dfs on index (of state) then reset it to flatten it back out 
leisure = pd.concat(dfs, join='outer', axis=1).fillna(nan_value).reset_index()
leisure.head()

,State,Coaster_ct,coaster_rank,brewery_count,brew_count_rank,brewery_econ_impact,brew_econ_impact_rank,brewery_production,brew_production_rank,art_museum_ct,art_museum_rank,orchestra_ct,orch_rank,Hist_site_ct,hist_rank,sport_ct,sport_rank
0,Alabama,9.0,25.0,52,39,777,28,88683,39,10,25,5.0,18.0,39,23,7.0,34.0
1,Arizona,6.0,35.0,121,26,1209,22,215051,26,13,19,3.0,27.0,47,14,15.0,11.0
2,Arkansas,7.0,31.0,46,45,496,37,51670,45,4,42,3.0,27.0,17,42,2.0,49.0
3,California,117.0,1.0,931,1,9031,1,3499182,1,96,2,31.0,1.0,147,47,69.0,1.0
4,Colorado,19.0,13.0,428,8,2451,8,946370,8,10,25,5.0,18.0,26,33,15.0,11.0


In [35]:
#convert df to numeric
numeric=leisure.columns.drop('State')
leisure[numeric]=leisure[numeric].apply(pd.to_numeric)

In [38]:
# List of all rankings
ranks= ['coaster_rank', 'brew_count_rank', 'art_museum_rank', 'orch_rank', 'hist_rank', 'sport_rank']
# Sum them up to get a score for each state
leisure['leisure_sum']= leisure[ranks].sum(axis=1)
# rank these scores
leisure['total_leisure_score']= leisure['leisure_sum'].rank(method='min', na_option='bottom').astype(int)

In [39]:
leisure

,State,Coaster_ct,coaster_rank,brewery_count,brew_count_rank,brewery_econ_impact,brew_econ_impact_rank,brewery_production,brew_production_rank,art_museum_ct,art_museum_rank,orchestra_ct,orch_rank,Hist_site_ct,hist_rank,sport_ct,sport_rank,leisure_sum,total_leisure_score
0,Alabama,9.0,25.0,52,39,777,28,88683,39,10,25,5.0,18.0,39,23,7.0,34.0,164.0,31
1,Arizona,6.0,35.0,121,26,1209,22,215051,26,13,19,3.0,27.0,47,14,15.0,11.0,132.0,24
2,Arkansas,7.0,31.0,46,45,496,37,51670,45,4,42,3.0,27.0,17,42,2.0,49.0,236.0,49
3,California,117.0,1.0,931,1,9031,1,3499182,1,96,2,31.0,1.0,147,47,69.0,1.0,53.0,6
4,Colorado,19.0,13.0,428,8,2451,8,946370,8,10,25,5.0,18.0,26,33,15.0,11.0,108.0,16
5,Connecticut,9.0,25.0,119,21,758,29,325416,21,18,14,3.0,27.0,64,8,7.0,34.0,129.0,23
6,Florida,65.0,3.0,374,3,4142,5,1332659,3,49,3,8.0,11.0,47,14,38.0,4.0,38.0,2
7,Georgia,28.0,11.0,155,14,1967,12,550178,14,19,13,7.0,12.0,50,11,18.0,10.0,71.0,8
8,Idaho,7.0,31.0,83,38,414,39,102390,38,1,51,0.0,0.0,10,51,3.0,45.0,216.0,45
9,Illinois,31.0,8.0,299,16,2984,7,438534,16,29,7,14.0,4.0,89,1,26.0,5.0,41.0,4


In [40]:
leisure.to_json('../json_out/leisure.json', orient='records')